In [5]:
pip install sqlalchemy

SyntaxError: invalid syntax (1042588720.py, line 1)

In [6]:
pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 17.5 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import sqlalchemy as sa
import requests

In [8]:
resource_id = '23172a73-7b85-49bd-9064-d600d2b21d37'
limit = 1500
offset = 0


Database Title: Public sector salary disclosure 2020

The database selected contains the sector, first names, last names, salaries, benefits, employer, and job title of all public sector and seconded employees in the year 2020

resource_id = '23172a73-7b85-49bd-9064-d600d2b21d37'
limit = 1500
offset = 0

api_url = 'https://data.ontario.ca/api/3/action/datastore_search?resource_id={}&limit={}&offset={}'.format(resource_id, limit, offset)

print(api_url)

api_response = requests.get(api_url)
api_response

data = api_response.json()
data

data['result']['records']

salary_data = pd.DataFrame(data['result']['records'])
salary_data

# Data Engineering

null_counts = salary_data.isnull().sum()
print(null_counts)

salary_data['full_name'] = salary_data['First name'] + ' ' + salary_data['Last name']
salary_data['full_name']

# Remove '$' and ',' from 'Ontario Commitment' and 'Total Project Costs' columns
salary_data['Salary'] = salary_data['Salary'].str.replace('[$,]','', regex=True)
salary_data['Benefits'] = salary_data['Benefits'].str.replace('[$,]','', regex=True)

salary_data['Salary'] = pd.to_numeric(salary_data['Salary'])
salary_data['Benefits'] = pd.to_numeric(salary_data['Benefits'])

salary_data['total_pay'] = salary_data['Salary'] + salary_data['Benefits']
salary_data['total_pay']

salary_data

relevant_columns = ['full_name', 'total_pay']
clean_total_pay = salary_data[relevant_columns]
clean_total_pay

# Push Data to Postgres Database

db_secret = {
    'drivername': 'postgresql+psycopg2',
    'host'      : 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port'      : '5432',
    'username'  : 'jreid9',
    'password'  : '2023!Schulich',
    'database'  : 'jreid9_db'
}

db_connection_url = sa.engine.URL.create(
    drivername = db_secret['drivername'],
    username   = db_secret['username'],
    password   = db_secret['password'],
    host       = db_secret['host'],
    port       = db_secret['port'],
    database   = db_secret['database']
)

db_connection_url

engine = sa.create_engine(db_connection_url)

with engine.connect() as connection:
    connection.execute('CREATE SCHEMA IF NOT EXISTS pipeline')

clean_total_pay.to_sql(
    name = 'total_pay', 
    schema = 'pipeline',
    con = engine,
    if_exists = 'replace',
    index = False,
    method = 'multi',
    dtype = {
        'full_name'  : sa.types.VARCHAR(50),
        'total_pay'  : sa.types.DECIMAL(10,2)
    }
)